In [12]:
#PAQUETES

import pandas as pd
import re
import numpy as np
## ALGOLIA
from algoliasearch.search_client import SearchClient
client = SearchClient.create("XD7RZG62BF","21e90530eb95b823eaa2d88154123b2b")
index = client.init_index('boot_camp')
## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from unidecode import unidecode


In [2]:
cv = pd.read_excel("random_sample_500.xlsx")
bc = pd.read_excel("ListBootcamps.xlsx")

In [3]:
cv.columns[0:30]

Index(['about', 'certifications', 'company', 'company_name_complete',
       'education', 'first_name', 'id', 'industry', 'interests', 'job_history',
       'job_title', 'job_title_complete', 'languages', 'last_name',
       'linkedin_id', 'linkedin_profile_url', 'location', 'middle_name',
       'num_connections', 'patents', 'publications', 'recommendations',
       'scraping_services_id', 'skills', 'snapshots', 'titles', 'updated',
       'v_extension', 'volunteered', 'Unnamed: 29'],
      dtype='object')

In [4]:
cv = cv[['certifications', 'education', 'job_history', 'job_title']]
cv.head()

,certifications,education,job_history,job_title
0,NaN,[{'degrees': ['Bachelor of Science in Accounta...,"[{'job_title': 'Asst Manager', 'company_name':...",Asst Manager
1,New Delhi. Nikhil completed his Bachelor of Te...,NaN,NaN,NaN
2,[{'certificate_authority': 'Sri Sathya Sai Ins...,[{'degrees': ['B.A (Law) B.Proc Hons ( Pol Sc....,"[{'job_title': 'LEGAL CONSULTANT', 'descriptio...",Executive Legal Director
3,Johannesburg. Currently responsible for manag...,NaN,NaN,NaN
4,science and technology studies and general ph...,more generally,NaN,NaN


In [5]:
cv["job_title"][0]

'Asst Manager'

# PRE-ANÁLISIS

In [6]:
#MIRAR REGEX DE LOS BOOTCAMPS
bc

,Bootcamp,Keywords,Filter by current location
0,4Geeks,‘4[ -]?Geeks’,"Chile, Venezuela"
1,Acámica,‘ac.mica’,"Argentina, Colombia, Mexico"
2,Ada ITW,‘ada.?itw|ada’,NaN
3,Bedu Tech,‘Bedu[ -]?Tech|bedu’,NaN
4,Bogotá Dev,‘bogot.[ -]?dev| bogot.[ -]?bootcamp’,NaN
5,Bootcamp Institute,‘bootcamp[ -]?institute’,NaN
6,Coderise International,‘coderise|coderise.?international’,NaN
7,Desafío Latam,‘desaf.o[ -]?latam’,NaN
8,Dev F.,'Dev ?F\.',NaN
9,Digital House,‘digital[ -]?house’,NaN


Se evidencia que probablemente muchas de la Regex de los bootcamps estén mal especificas y existe la posibilidad de que existan falsos positivos. Procedamos a hacer la busqueda con estas Regex.

# BUSQUEDA PRELIMINAR DE MATCHS CON REGEX ORIGINALES

# Educación

In [7]:
##EXTRAIGO LA EDUCACIÓN

E = []
for i in cv["education"]:
    E.append(str(i).lower()) #creo una lista con la educación de cada participante y la convierto en string


K = []

for i in bc["Keywords"]:
    i = i.replace("‘", ""); i = i.replace("’", "") 
    K.append(str(i).lower()) #creo una lista con las Regex de los bootcamps a la vez que: 1) se convierten en minuscula y 2)elimino los caracteres "’" y "‘", para poder hacer la busqueda

In [8]:
#LAS DOS LISTAS SE VEN ASÍ:
K[0:10]

['4[ -]?geeks',
 'ac.mica',
 'ada.?itw|ada',
 'bedu[ -]?tech|bedu',
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 "'dev ?f\\.'",
 'digital[ -]?house']

In [9]:
E[5:20]

['nan',
 '[{\'degrees\': [\'directrice de casting\'], \'education_org\': \'cefpf centre européen de formation de films\', \'summary\': "savoir organiser un casting.\\n s\'adapter à tous types de films et de tournages.\\n analyser un scénario pour trouver les personnages.\\n casting sauvage.\\n maîtrise de la caméra.\\n montage des essais comédiens pour le réalisateur."}, {\'degrees\': [\'danseuse classique, niveau supérieur\'], \'education_org\': \'conservatoire de danse à nancy\'}, {\'degrees\': [\'jazz\'], \'education_org\': \'cours de danse avec patrice valéro, rédha...\'}, {\'degrees\': [\'comédienne\'], \'education_org\': \'théâtre : cours virio ,studio pygmalion\'}]',
 "[{'start': '2004', 'education_org': 'school of pedagogical and technological education', 'summary': 'degree title:\\n «educator technologist mechanical engineer»\\n thesis: gear failures'}]",
 'nan',
 "{'size_max': 500, 'address_hq': '451 wolf fork rd dayton, washington 99328-8768 united states', 'location': {'sta

In [18]:
##EXTRAIGO LOS NOMBRES DE LOS CENTROS EDUCATIVOS: creo una lista de listas, que contenga la listas de instituciones educativas.

educ = []
for e,i in zip(E, range(0,502)):
    print(e)
    A = re.findall("'education_org': '.*?'", e) #con est regex extraigo todas las instituciones educativas, tal que conserve cualquier caracter hasta el "'".
    for i in range(len(A)):
        A[i] = A[i].replace("'education_org': '", "") #elimino el titulo de education_org
        A[i] = A[i].replace("'", "") #elimino "'"
    ## no se agregan los nulos
    if len(A)>0: 
       
        educ.append(A) #agrego

        
    

es'], 'end': '2013', 'start': '2013', 'education_org': 'hospital das clinicas da usp - iot', 'summary': 'activities and societies: atividades cirúrgicas, ambulatoriais e práticas laboratoriais'}, {'degrees': ['cirurgia da mão, cirurgia da mão e microcirurgia', 'cirurgia da mão, cirurgia da mão e microcirurgia'], 'end': '2013', 'start': '2011', 'education_org': 'hospital de clinicas do paraná - ufpr', 'summary': 'activities and societies: residência médica'}, {'degrees': ['ortopedista, ortopedia e traumatologia', 'ortopedista, ortopedia e traumatologia'], 'end': '2010', 'start': '2008', 'education_org': 'hospital santa casa de misericordia de curitiba - pr', 'summary': 'activities and societies: residência médica'}, {'degrees': ['médica, medicina', 'médica, medicina'], 'end': '2007', 'start': '2003', 'education_org': 'escola de medicina da santa casa de vitoria - es', 'summary': 'activities and societies: graduação em medicina'}, {'degrees': ['medicina, medicina', 'medicina, medicina'],

In [11]:
#la lista de educación se ve así
educ[0:10]

[['ateneo de naga university', 'worldtech resources institute'],
 ['stanger high'],
 ['cefpf centre européen de formation de films',
  'conservatoire de danse à nancy',
  'cours de danse avec patrice valéro, rédha...',
  'théâtre : cours virio ,studio pygmalion'],
 ['school of pedagogical and technological education'],
 ['inservice a opleiding groningen', 'mulo bedum', 'nlp'],
 ['faculdade de tecnologia e ciências', 'intituto superior técnico'],
 ['universidade de évora'],
 ['academia mac', 'academia mac', 'duoc uc'],
 ['worldtech information solutions', 'notre dame of dadiangas univeristy'],
 ['universidad complutense de madrid']]

In [18]:
## fuzzy test
r1 = fuzz.ratio("hola", 'a')
val = "Acámica"
rpartial =fuzz.partial_ratio(val,"acamica" )
rpartial2 =fuzz.partial_ratio(unidecode(val),"acamica" )

val = val.lower()
val = unidecode(val)

print(rpartial)
print(val)
print(rpartial2)

71
acamica
86


In [14]:
#VIASUALIZEMOS LAS COICIDENCIAS POR KEYWORD, INSTITUTO Y TITULO DE TRABAJO
for k in K:
    for n in range(len(educ)):
        for h in educ[n]:
            A = re.findall(k, h)
            if len(A)>=1:
                print(k, " -- ", h,"--", cv["job_title"][n]) 

ac.mica  --  codecademy/acamica -- Senior Javascript Developer
ada.?itw|ada  --  enform (safety association for canada’s upstream oil and gas industry) -- Senior Operations Manager
ada.?itw|ada  --  enform (safety association for canada’s upstream oil and gas industry) -- Senior Operations Manager
ada.?itw|ada  --  enform (safety association for canada’s upstream oil and gas industry) -- Senior Operations Manager
ada.?itw|ada  --  enform (safety association for canada’s upstream oil and gas industry) -- Senior Operations Manager
ada.?itw|ada  --  universidad nacional experimental politecnica de la fuerza armada nacional -- Gestor de Cobros
ada.?itw|ada  --  saúde oral - implantologia especializada -- Diretora clínica
ada.?itw|ada  --  saúde oral - implantologia especializada -- Diretora clínica
ada.?itw|ada  --  faculdades integradas de aracruz -- nan
bedu[ -]?tech|bedu  --  mulo bedum -- Hospital & Health Care Professional
bedu[ -]?tech|bedu  --  mavo, bedum -- Assistent Controller
be

Se observan varios FP con la Regex orginales EJ:

1. nivel[ -]?pro  --  curso de inglés nivel proficiency y curso de business english, chichester college, reino unido
2. world[ -]?tech|world[ -]?tech[ -]?inc| world[ -]?tech[ -]?makers|wtm '  --  world-tech resources institute


 Vale la pena modificarlas. En especial las de: Ada ITW, Bedu Tech, Laboratoria, Le Wagon, Nivel Pro, Pygmalion y World Tech Inc. 
 El caso de pygmalion fue eliminada debido a que ninguna parecia ser de algún programador o perfil dedicado a la programación.
 
Además en el caso Noukoud, Bictia, Muktek e Ijalti no tienen asignada regex, vale hacer una.

In [15]:
K =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw', #eliminando el "ada" de "ada.?itw|ada" se puede mejorar el desempeño de la regex. 
 'bedu[ -]?tech', #mismo caso amterior
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 "'dev ?f\\.'",
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 "'hola.?code'",
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', #este está bien así
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', #con aplicarle un espacio al final se garantiza que palabras como laboratian o laboratorial no coincida. Sin embargo sigue teniendo cierto margen de error.
 'le.?wagon', #mismo caso inicial
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', #mpdificación
 'nivel[ -]?pro ', #mismo cambio del espacio al final que laboratoria.
 'noukod|noukood|noukd', #modificación
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'reprograma',
 "worldtech", 
 'bictia|bi.?tia', #modificación
 'talento[ -]?digital']

In [16]:
#VIASUALIZEMOS LAS COICIDENCIAS POR KEYWORD, INSTITUTO Y TITULO DE TRABAJO
ids = []
for k in K:
    for n in range(len(educ)):
        for h in educ[n]:
            A = re.findall(k, h)
            if len(A)>=1:
                print(k, " -- ", h,"--", cv["job_title"][n], f"id {n}")
                ids.append((n, k, h, 1))#guardo id, keyword y 1 para educación

ac.mica  --  codecademy/acamica -- Senior Javascript Developer id 413
desaf.o[ -]?latam  --  desafio latam -- nan id 27
digital[ -]?house  --  digital house -- Social Media Intelligence & Marketing Research id 430
hack.?academy  --  biohack academy -- nan id 60
iron.?hack  --  ironhack -- Head of Strategy id 450
keep.?coding  --  keepcoding -- Ingeniero I+D+i id 70
plataforma[ -]?5  --  plataforma 5 - coding school -- Business Data Science Consultant id 72
reprograma  --  reprograma -- nan id 407
worldtech  --  worldtech resources institute -- Asst Manager id 0
worldtech  --  worldtech information solutions -- I create Lead generating and Conversion focused Landing Page for service based business id 18
worldtech  --  worldtech resources institute -- Frontliner id 284
worldtech  --  worldtech -- Freelance Developer id 380
worldtech  --  worldtech -- Freelance Developer id 380


In [17]:
ids

[(413, 'ac.mica', 'codecademy/acamica', 1),
 (27, 'desaf.o[ -]?latam', 'desafio latam', 1),
 (430, 'digital[ -]?house', 'digital house', 1),
 (60, 'hack.?academy', 'biohack academy', 1),
 (450, 'iron.?hack', 'ironhack', 1),
 (70, 'keep.?coding', 'keepcoding', 1),
 (72, 'plataforma[ -]?5', 'plataforma 5 - coding school', 1),
 (407, 'reprograma', 'reprograma', 1),
 (0, 'worldtech', 'worldtech resources institute', 1),
 (18, 'worldtech', 'worldtech information solutions', 1),
 (284, 'worldtech', 'worldtech resources institute', 1),
 (380, 'worldtech', 'worldtech', 1),
 (380, 'worldtech', 'worldtech', 1)]

Siguen habiendo algunos FP

# Certificados

In [18]:
C = []
for i in cv["certifications"]:
    C.append(str(i).lower()) #creo una lista con la certificación de cada participante y la convierto en string

In [19]:
##EXTRAIGO LOS NOMBRES DE LA ENTIDAD QUE EXPIDIÓ EL CERTIFICADO: creo una lista de listas, que contenga la listas certificados.

cert = []
for e,i in zip(C, range(0,502)):
    A = re.findall("'certificate_authority': '.*?'", e) #con est regex extraigo todas las certificaciones, tal que conserve cualquier caracter hasta el "'".
    for i in range(len(A)):
        A[i] = A[i].replace("'certificate_authority': '", "") #elimino el titulo de certificate_authority
        A[i] = A[i].replace("'", "") #elimino "'"
    cert.append(A) #agrego la lista

In [20]:
for k in K:
    for n in range(len(cert)):
        for h in cert[n]:
            A = re.findall(k, h)
            if len(A)>=1:
                print(k, " -- ", h,"--", cv["job_title"][n], f"id {n}")
                ids.append((n, k, h, 0)) #guardo id, keyword y para certificación

ac.mica  --  acamica -- Software Engineer id 26
ac.mica  --  acamica -- Software Engineer id 26
ac.mica  --  acamica -- Software Engineer id 26
icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding  --  icreate consulting -- nan id 453
keep.?coding  --  keepcoding -- Profesor en Formación Permanente. id 251
keep.?coding  --  keepcoding -- Security Analyst id 274
keep.?coding  --  keepcoding -- Java Software Engineer id 369
keep.?coding  --  keepcoding -- Java Software Engineer id 369
keep.?coding  --  keepcoding -- Software Architect id 479
keep.?coding  --  keepcoding -- .Net Software Developer id 497


A simple vista solo parece haber un solo FP

In [21]:
ids

[(413, 'ac.mica', 'codecademy/acamica', 1),
 (27, 'desaf.o[ -]?latam', 'desafio latam', 1),
 (430, 'digital[ -]?house', 'digital house', 1),
 (60, 'hack.?academy', 'biohack academy', 1),
 (450, 'iron.?hack', 'ironhack', 1),
 (70, 'keep.?coding', 'keepcoding', 1),
 (72, 'plataforma[ -]?5', 'plataforma 5 - coding school', 1),
 (407, 'reprograma', 'reprograma', 1),
 (0, 'worldtech', 'worldtech resources institute', 1),
 (18, 'worldtech', 'worldtech information solutions', 1),
 (284, 'worldtech', 'worldtech resources institute', 1),
 (380, 'worldtech', 'worldtech', 1),
 (380, 'worldtech', 'worldtech', 1),
 (26, 'ac.mica', 'acamica', 0),
 (26, 'ac.mica', 'acamica', 0),
 (26, 'ac.mica', 'acamica', 0),
 (453,
  'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
  'icreate consulting',
  0),
 (251, 'keep.?coding', 'keepcoding', 0),
 (274, 'keep.?coding', 'keepcoding', 0),
 (369, 'keep.?coding', 'keepcoding', 0),
 (369, 'keep.?coding', 'keepcoding', 0),
 (479, 'ke

# POTENCIALES MATCH

In [22]:
#POTENCIALES PERFILES
#ELIMINAR DUPLICADOS
ids = list(set(ids))
ids

[(453,
  'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
  'icreate consulting',
  0),
 (0, 'worldtech', 'worldtech resources institute', 1),
 (430, 'digital[ -]?house', 'digital house', 1),
 (479, 'keep.?coding', 'keepcoding', 0),
 (251, 'keep.?coding', 'keepcoding', 0),
 (60, 'hack.?academy', 'biohack academy', 1),
 (413, 'ac.mica', 'codecademy/acamica', 1),
 (26, 'ac.mica', 'acamica', 0),
 (18, 'worldtech', 'worldtech information solutions', 1),
 (27, 'desaf.o[ -]?latam', 'desafio latam', 1),
 (380, 'worldtech', 'worldtech', 1),
 (497, 'keep.?coding', 'keepcoding', 0),
 (274, 'keep.?coding', 'keepcoding', 0),
 (369, 'keep.?coding', 'keepcoding', 0),
 (72, 'plataforma[ -]?5', 'plataforma 5 - coding school', 1),
 (407, 'reprograma', 'reprograma', 1),
 (284, 'worldtech', 'worldtech resources institute', 1),
 (70, 'keep.?coding', 'keepcoding', 1),
 (450, 'iron.?hack', 'ironhack', 1)]

In [23]:
#DATAFRAME CON LOS MATCHS

lista_id = []
for i in ids:
    lista_id.append(i[0])

cvf = cv.iloc[lista_id]
cvf.reset_index(inplace = True)

In [24]:
cvf

,index,certifications,education,job_history,job_title
0,453,"[{'certificate_authority': 'Citeforma', 'title...",[{'degrees': ['3º Grau Profissional (Equivalên...,"[{'company_intern_id': 'icreate-consulting', '...",NaN
1,0,NaN,[{'degrees': ['Bachelor of Science in Accounta...,"[{'job_title': 'Asst Manager', 'company_name':...",Asst Manager
2,430,"[{'title': 'Language Certifications', 'end': '...","[{'degrees': ['Digital Marketing, Analytics, W...","[{'company_intern_id': 'prune', 'job_title': '...",Social Media Intelligence & Marketing Research
3,479,"[{'certificate_authority': 'KeepCoding', 'titl...",[{'degrees': ['Ingeniero Técnico en Informátic...,"[{'company_intern_id': 'viewnext', 'location':...",Software Architect
4,251,"[{'certificate_authority': 'Scrum Alliance', '...",[{'degrees': ['Grado en Ingeniería de Informát...,"[{'company_intern_id': 'moelia-development', '...",Profesor en Formación Permanente.
5,60,[{'certificate_authority': 'APMG-International...,"[{'end': '2016', 'start': '2016', 'education_o...",NaN,NaN
6,413,NaN,[{'degrees': ['Faculty of Engineering -PHP y M...,"[{'company_intern_id': 'avalith-net', 'job_tit...",Senior Javascript Developer
7,26,[{'certificate_authority': 'Fundacion Proydesa...,"[{'degrees': [""Engineer's degree, Software Eng...","[{'company_intern_id': 'globant', 'start': 'Ap...",Software Engineer
8,18,"[{'certificate_authority': 'Cisco', 'title': '...","[{'degrees': ['CCNA Training, ICND1, System, N...",[{'company_intern_id': 'mercedes-benz-group-se...,I create Lead generating and Conversion focuse...
9,27,"[{'certificate_authority': 'Code School', 'tit...","[{'degrees': ['Junior Web Developer', 'Junior ...",[{'company_intern_id': 'centro-de-energia-fcfm...,NaN


In [25]:
#crear variable de bootcamp y dummies sobre certificación o educación

cvf["bootcamp"] = 0
for i in range(len(ids)):
    cvf["bootcamp"][i] = ids[i][2]


cvf["C"] = 2
for i in range(len(ids)):
    cvf["C"][i] = ids[i][3]


cvf["certifications"] = cvf["certifications"].str.lower()
cvf["education"] = cvf["education"].str.lower()

<ipython-input-25-edd44507d34c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cvf["bootcamp"] = 0
<ipython-input-25-edd44507d34c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cvf["bootcamp"][i] = ids[i][2]
C:\Users\War\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\W

In [26]:
#año graduado según columna educación

cvf["año_graduación"] = 0

for i in range(len(ids)):
    if cvf["C"][i] == 1:
        cvf["año_graduación"][i] = cvf["education"][i].split(ids[i][2])[0][::-1].split("dne")[0][::-1][4:8]
        # cvf["año_graduación"][i] = cvf["certifications"][i].split(ids[i][2])[0][::-1].split("dne")[0][::-1][3:14]

<ipython-input-26-24797ff9a557>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cvf["año_graduación"] = 0
<ipython-input-26-24797ff9a557>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cvf["año_graduación"][i] = cvf["education"][i].split(ids[i][2])[0][::-1].split("dne")[0][::-1][4:8]


In [27]:
#año graduado según columna certificación

for i in range(len(ids)):
    if cvf["C"][i] == 0:
        # cvf["año_graduación"][i] = cvf["education"][i].split(ids[i][2])[0][::-1].split("dne")[0][::-1][4:8]
        cvf["año_graduación"][i] = cvf["certifications"][i].split(ids[i][2])[0][::-1].split("dne")[0][::-1][3:14]

<ipython-input-27-1b8b0230a5ec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cvf["año_graduación"][i] = cvf["certifications"][i].split(ids[i][2])[0][::-1].split("dne")[0][::-1][3:14]


In [28]:
cvf.to_excel("boot_final.xlsx")

In [29]:
#base de matchs
cvf

,index,certifications,education,job_history,job_title,bootcamp,C,año_graduación
0,453,"[{'certificate_authority': 'citeforma', 'title...",[{'degrees': ['3º grau profissional (equivalên...,"[{'company_intern_id': 'icreate-consulting', '...",NaN,icreate consulting,0,"'present'},"
1,0,NaN,[{'degrees': ['bachelor of science in accounta...,"[{'job_title': 'Asst Manager', 'company_name':...",Asst Manager,worldtech resources institute,1,1995
2,430,"[{'title': 'language certifications', 'end': '...","[{'degrees': ['digital marketing, analytics, w...","[{'company_intern_id': 'prune', 'job_title': '...",Social Media Intelligence & Marketing Research,digital house,1,2017
3,479,"[{'certificate_authority': 'keepcoding', 'titl...",[{'degrees': ['ingeniero técnico en informátic...,"[{'company_intern_id': 'viewnext', 'location':...",Software Architect,keepcoding,0,certificate
4,251,"[{'certificate_authority': 'scrum alliance', '...",[{'degrees': ['grado en ingeniería de informát...,"[{'company_intern_id': 'moelia-development', '...",Profesor en Formación Permanente.,keepcoding,0,"'present',"
5,60,[{'certificate_authority': 'apmg-international...,"[{'end': '2016', 'start': '2016', 'education_o...",NaN,NaN,biohack academy,1,2016
6,413,NaN,[{'degrees': ['faculty of engineering -php y m...,"[{'company_intern_id': 'avalith-net', 'job_tit...",Senior Javascript Developer,codecademy/acamica,1,2014
7,26,[{'certificate_authority': 'fundacion proydesa...,"[{'degrees': [""engineer's degree, software eng...","[{'company_intern_id': 'globant', 'start': 'Ap...",Software Engineer,acamica,0,"'present',"
8,18,"[{'certificate_authority': 'cisco', 'title': '...","[{'degrees': ['ccna training, icnd1, system, n...",[{'company_intern_id': 'mercedes-benz-group-se...,I create Lead generating and Conversion focuse...,worldtech information solutions,1,2012
9,27,"[{'certificate_authority': 'code school', 'tit...","[{'degrees': ['junior web developer', 'junior ...",[{'company_intern_id': 'centro-de-energia-fcfm...,NaN,desafio latam,1,2015
